In [47]:
import os
import sys
import pandas as pd
import io
import numpy as np
import geopandas as gpd

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
from scripts.utils.file_helpers import (
    pull_csv_from_directory, upload_csv_aws
) 

In [48]:
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/climate_risk/sea_level_rise/loss/climate_central/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'RCP_wetland_data.csv'


In [49]:
wetland_data = pd.read_csv('RCP_wetland_data.csv')


In [50]:
wetland_data = pd.read_csv('RCP_wetland_data.csv')

# Adjust the data
adjusted_wetland_data = wetland_data[22:]

# Set the first row as the header and reset index
adjusted_wetland_data.columns = adjusted_wetland_data.iloc[0]
adjusted_wetland_data = adjusted_wetland_data[1:].reset_index(drop=True)

# Drop the index column if it has been set as a column
adjusted_wetland_data.reset_index(drop=True, inplace=True)

# Rename columns to ensure no extra index is included
adjusted_wetland_data.columns.name = None

# Filter columns explicitly
columns_to_keep = [col for col in adjusted_wetland_data.columns 
                    if 'County' in col or '2020' in col or '2100' in col]
adjusted_wetland_data = adjusted_wetland_data[columns_to_keep]

adjusted_wetland_data.columns

Index(['County', 'RCP_2.6__5th_percentile__2020',
       'RCP_2.6__50th_percentile__2020', 'RCP_2.6__95th_percentile__2020',
       'RCP_4.5__5th_percentile__2020', 'RCP_4.5__50th_percentile__2020',
       'RCP_4.5__95th_percentile__2020', 'RCP_8.5__5th_percentile__2020',
       'RCP_8.5__50th_percentile__2020', 'RCP_8.5__95th_percentile__2020',
       'RCP_2.6__5th_percentile__2100', 'RCP_2.6__50th_percentile__2100',
       'RCP_2.6__95th_percentile__2100', 'RCP_4.5__5th_percentile__2100',
       'RCP_4.5__50th_percentile__2100', 'RCP_4.5__95th_percentile__2100',
       'RCP_8.5__5th_percentile__2100', 'RCP_8.5__50th_percentile__2100',
       'RCP_8.5__95th_percentile__2100'],
      dtype='object')

In [77]:
adjusted_wetland_data.head()

,County,RCP_2.6__5th_percentile__2020,RCP_2.6__50th_percentile__2020,RCP_2.6__95th_percentile__2020,RCP_4.5__5th_percentile__2020,RCP_4.5__50th_percentile__2020,RCP_4.5__95th_percentile__2020,RCP_8.5__5th_percentile__2020,RCP_8.5__50th_percentile__2020,RCP_8.5__95th_percentile__2020,RCP_2.6__5th_percentile__2100,RCP_2.6__50th_percentile__2100,RCP_2.6__95th_percentile__2100,RCP_4.5__5th_percentile__2100,RCP_4.5__50th_percentile__2100,RCP_4.5__95th_percentile__2100,RCP_8.5__5th_percentile__2100,RCP_8.5__50th_percentile__2100,RCP_8.5__95th_percentile__2100
0,Alameda,40.1,40.1,39.8,40.1,40.0,39.8,40.1,40.0,39.8,40.1,39.9,33.4,40.1,38.6,31.3,39.9,35.8,24.5
1,Alpine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Butte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Calaveras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# Function to calculate percent change between 2020 and 2100 columns
def calculate_percent_change(data, leave_alone=[]):
    # Convert columns to numeric, forcing non-numeric to NaN (skip columns in leave_alone)
    numeric_data = data.copy()
    for col in data.columns:
        if col not in leave_alone:
            numeric_data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Define columns for 2020 and 2100
    cols_2020 = [col for col in numeric_data.columns if '2020' in col]
    cols_2100 = [col for col in numeric_data.columns if '2100' in col]
    
    # Calculate percent change
    percent_change = pd.DataFrame()

    for col_2020 in cols_2020:
        # Find the matching 2100 column
        col_2100 = col_2020.replace('2020', '2100')

        if col_2100 in cols_2100:
            # Calculate percent change, handling NaN values
            percent_change[col_2020 + '_to_' + col_2100] = (
                (numeric_data[col_2100] - numeric_data[col_2020]) / numeric_data[col_2020]
            ) * 100
    
    # Concatenate the percent change DataFrame with the original numeric data
    result = pd.concat([numeric_data, percent_change], axis=1)
    
    return result

# Function to rename columns, allowing some to be left unchanged
def rename_columns(data, leave_alone=[]):
    def rename_column(col):
        if col in leave_alone:
            return col
        words = col.split('_')
        return '_'.join(words[:4]) + '_percent_change'
    
    # Apply renaming function to columns
    data.columns = [rename_column(col) for col in data.columns]
    return data

# List of columns to leave unchanged
column_leave_alone = ['County']

In [93]:
# Run the calculation and renaming
adjusted_wetland_metric = calculate_percent_change(adjusted_wetland_data, leave_alone=column_leave_alone)

# Filter for columns that contain 'County' or 'to'
filtered_columns = [col for col in adjusted_wetland_metric.columns if 'County' in col or 'to' in col]

# Create a new DataFrame with only the filtered columns
filtered_wetland_metric = adjusted_wetland_metric[filtered_columns]
# Remove duplicate columns
filtered_wetland_metric = filtered_wetland_metric.loc[:, ~filtered_wetland_metric.columns.duplicated()]

wetland_metric_percent_change = rename_columns(filtered_wetland_metric, leave_alone=column_leave_alone)

wetland_metric_percent_change.columns = wetland_metric_percent_change.columns.str.lower()
wetland_metric_percent_change = wetland_metric_percent_change.applymap(lambda s: s.lower() if type(s) == str else s)

# Display the resulting DataFrame
wetland_metric_percent_change

C:\Users\jespi\AppData\Local\Temp\ipykernel_11312\1699519615.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  wetland_metric_percent_change = wetland_metric_percent_change.applymap(lambda s: s.lower() if type(s) == str else s)


,county,rcp_2.6__5th_percent_change,rcp_2.6__50th_percent_change,rcp_2.6__95th_percent_change,rcp_4.5__5th_percent_change,rcp_4.5__50th_percent_change,rcp_4.5__95th_percent_change,rcp_8.5__5th_percent_change,rcp_8.5__50th_percent_change,rcp_8.5__95th_percent_change
0,alameda,0.0,-0.498753,-16.080402,0.0,-3.500000,-21.356784,-0.498753,-10.500000,-38.442211
1,alpine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,amador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,butte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,calaveras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,colusa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,contra costa,0.0,-0.857143,-27.745665,0.0,-6.876791,-34.202899,-0.857143,-20.057307,-57.803468
7,del norte,0.0,0.000000,-10.714286,0.0,0.000000,-14.285714,0.000000,-7.142857,-28.571429
8,el dorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,fresno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# read in CA census tiger file
ca_tract_county = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_tract_county = gpd.read_file(ca_tract_county)
ca_tract_county = ca_tract_county.drop(columns={'field_1', 'geometry'})
ca_tract_county.columns = ca_tract_county.columns.str.lower()
ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)

ca_tract_county

C:\Users\jespi\AppData\Local\Temp\ipykernel_11312\3785741718.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ca_tract_county = ca_tract_county.applymap(lambda s: s.lower() if type(s) == str else s)


,tract,countyfp,county
0,06085504321,085,santa clara
1,06085504410,085,santa clara
2,06085507003,085,santa clara
3,06085507004,085,santa clara
4,06085502204,085,santa clara
...,...,...,...
9124,06059001303,059,orange
9125,06059001304,059,orange
9126,06059001401,059,orange
9127,06013367200,013,contra costa


In [94]:
wetland_metric_merge = pd.merge(ca_tract_county, wetland_metric_percent_change, on='county', how='left')
wetland_metric_merge

,tract,countyfp,county,rcp_2.6__5th_percent_change,rcp_2.6__50th_percent_change,rcp_2.6__95th_percent_change,rcp_4.5__5th_percent_change,rcp_4.5__50th_percent_change,rcp_4.5__95th_percent_change,rcp_8.5__5th_percent_change,rcp_8.5__50th_percent_change,rcp_8.5__95th_percent_change
0,06085504321,085,santa clara,0.0,-0.970874,-13.725490,0.0,-4.854369,-15.686275,-0.970874,-10.679612,-23.529412
1,06085504410,085,santa clara,0.0,-0.970874,-13.725490,0.0,-4.854369,-15.686275,-0.970874,-10.679612,-23.529412
2,06085507003,085,santa clara,0.0,-0.970874,-13.725490,0.0,-4.854369,-15.686275,-0.970874,-10.679612,-23.529412
3,06085507004,085,santa clara,0.0,-0.970874,-13.725490,0.0,-4.854369,-15.686275,-0.970874,-10.679612,-23.529412
4,06085502204,085,santa clara,0.0,-0.970874,-13.725490,0.0,-4.854369,-15.686275,-0.970874,-10.679612,-23.529412
...,...,...,...,...,...,...,...,...,...,...,...,...
9124,06059001303,059,orange,0.0,0.000000,-20.481928,0.0,-5.952381,-27.710843,0.000000,-15.476190,-55.421687
9125,06059001304,059,orange,0.0,0.000000,-20.481928,0.0,-5.952381,-27.710843,0.000000,-15.476190,-55.421687
9126,06059001401,059,orange,0.0,0.000000,-20.481928,0.0,-5.952381,-27.710843,0.000000,-15.476190,-55.421687
9127,06013367200,013,contra costa,0.0,-0.857143,-27.745665,0.0,-6.876791,-34.202899,-0.857143,-20.057307,-57.803468


In [95]:
wetland_metric_merge.to_csv('climate_wetland_loss_metric.csv')

Function Call

In [96]:
@append_metadata
def climate_slr_wetland_loss(input_csv, export=False, varname=''):
    '''
    Uploads the crop loss metrics to S3 bucket. The metric is:
    * Delta percent signal in wetland area lost between 2020 and 2100
    
    Data for this metric was sourced from Climate Central at:
    https://coastal.climatecentral.org/map/13/-117.0358/32.5017/?theme=wetlands&map_type=annual_accretion

    Methods
    -------
    Relevant columns to our data metrics were isolated.
    Percent change between estimated wetland area between 2020 and its 2100 counterpart were calculated.
    Metric columns were named, isolated, and merged to CA census tracts based on shared county.
    
    Parameters
    ----------
    input_csv: string
        csv wetland loss data 
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI wetland loss metrics to AWS
        True = will upload resulting df containing CAL CRAI wetland loss metrics to AWS

    Script
    ------
    climate_slr_wetland_loss.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: relevant columns were isolated (percentile data from 2020 and 2100)')
    print('Data transformation: percent change was calculated, isolated, and renamed.')
    print('Data transformation: data was merged with California census tracts.') 
 
    if export == True:
        bucket_name = 'ca-climate-index'
        directory = '3_fair_data/index_data'
        export_filename = [input_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{input_csv} uploaded to AWS.')
 
    if os.path.exists(input_csv):
        os.remove(input_csv)

In [97]:
input_csvs = [
            'climate_wetland_loss_metric.csv'
            ]

varnames = [
            'climate_climate_central_wetland_loss'
            ]

# Process the data and export
for input_csv, varname in zip(input_csvs, varnames):
    print(f'Processing {input_csv} with varname {varname}')
    climate_slr_wetland_loss(input_csv, export=True, varname='test')
    print(f'Completed uploading {input_csv} with varname {varname}!')

Processing climate_wetland_loss_metric.csv with varname climate_climate_central_wetland_loss
Completed uploading climate_wetland_loss_metric.csv with varname climate_climate_central_wetland_loss!
